In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
from progressbar import progressbar as pb
import itertools
import collections
import re
from sklearn.model_selection import train_test_split as TTS

In [3]:
import pandas as pd
import numpy as np
from progressbar import progressbar as pb
df = pd.read_csv('/content/drive/My Drive/Embeddings/big_data_lower.csv')
df.head()

,0,1,2,3,4,5
0,совещательный орган при императоре в начале xx в,сенат,государственный совет,консилия министров,верховный тайный совет,2
1,министр внутренних дел с 1904 г либерал автор ...,лорис меликов,святополк мирский,милюков,витте,2
2,какое из названных событий произошло 27 января...,подрыв флагманского корабля петропавловск,высадка японских войск на ляодунском полуострове,героический бой варяга и корейца,заключение портсмутского мирного договора,3
3,что из названного относится к результатам перв...,появление многопартийности,формирование конституционной монархии,ликвидация сословного строя,введение рабочего контроля за производством,1
4,творчество поэтов гумилева ахматовой мандельшт...,акмеизму,футуризму,импрессионизму,реализму,1


In [30]:
df2 = pd.read_csv('/content/drive/My Drive/Embeddings/big_data_labels.csv')
df['label'] = df2['label']
df.head()

,0,1,2,3,4,5,label
0,совещательный орган при императоре в начале xx в,сенат,государственный совет,консилия министров,верховный тайный совет,2,История
1,министр внутренних дел с 1904 г либерал автор ...,лорис меликов,святополк мирский,милюков,витте,2,История
2,какое из названных событий произошло 27 января...,подрыв флагманского корабля петропавловск,высадка японских войск на ляодунском полуострове,героический бой варяга и корейца,заключение портсмутского мирного договора,3,История
3,что из названного относится к результатам перв...,появление многопартийности,формирование конституционной монархии,ликвидация сословного строя,введение рабочего контроля за производством,1,История
4,творчество поэтов гумилева ахматовой мандельшт...,акмеизму,футуризму,импрессионизму,реализму,1,История


In [4]:
import re
import itertools
import collections
import numpy as np

def gen_bigrams():
    """
      Generates all bigrams for characters from `bigram_chars`
    """
    trigram_chars="0123456789абвгдеёжзийклмнопрстуфхцчшщъыьэюя"
    t3=[''.join(x) for x in itertools.product(trigram_chars,repeat=2)] #len(words)>=3
    t2_start=['#'+''.join(x) for x in itertools.product(trigram_chars)] #len(words)==2
    t2_end=[''.join(x)+'#' for x in itertools.product(trigram_chars)] #len(words)==2
    trigrams=t3+t2_start+t2_end
    vocab_size=len(trigrams)
    trigram_map=dict(zip(trigrams,range(1,vocab_size+1))) # trigram to index mapping, indices starting from 1
    return trigram_map
  
def sentences_to_bag_of_bigrams(sentences):
    """
      Converts a sentence to bag-of-trigrams
      `sentences`: list of strings
      `trigram_BOW`: return value, (len(sentences),len(trigram_map)) size array
    """
    trigram_map=gen_bigrams()
    trigram_BOW=np.zeros((len(sentences),len(trigram_map)+1)) # one row for each sentence
    filter_pat=r'[\!"#&\(\)\*\+,-\./:;<=>\?\[\\\]\^_`\{\|\}~\t\n]' # characters to filter out from the input
    for j,sent in enumerate(sentences):
        sent=re.sub(filter_pat, '', sent).lower() # filter out special characters from input
        sent=re.sub(r"(\s)\s+", r"\1", sent) # reduce multiple whitespaces to single whitespace
        words=sent.split(' ')
        indices=collections.defaultdict(int)
        for word in words:
            word='#'+word+'#'
            for k in range(len(word)-1): # generate all trigrams for word `word` and update `indices`
                trig=word[k:k+2]
                print(trig)
                idx=trigram_map.get(trig, 0)
                indices[idx]=indices[idx]+1
        for key,val in indices.items(): #covert `indices` dict to np array
            trigram_BOW[j,key]=val
    return trigram_BOW

In [5]:
sentences_to_bag_of_bigrams(['нейронная сеть'])

#н
не
ей
йр
ро
он
нн
на
ая
я#
#с
се
ет
ть
ь#


array([[0., 0., 0., ..., 0., 0., 1.]])

In [5]:
hashes = np.zeros((df.shape[0], df.shape[1]-1, 1936))

for i in pb(range(df.shape[0])):
  for j in range(df.shape[1]-1):
    hashes[i,j] = sentences_to_bag_of_bigrams([df.iloc[i,j]])[0]

100% (13865 of 13865) |##################| Elapsed Time: 0:00:55 Time:  0:00:55


In [6]:
hashes.shape

(13865, 5, 1936)

In [0]:
samples = hashes
target = np.array(df['5'])

X_train, X_test, y_train, y_test = TTS(hashes, target, test_size=0.25, random_state=4)

In [8]:
X_train.shape

(10398, 5, 1936)

In [0]:
import itertools
import re
import numpy as np
import collections

In [0]:
def get_batch(X_val, y_val, batch_sz=16):
    
    '''
    Разбиение выборки на батчи заданного размера.
    
    X_val - векторное представление данных
    y_val - разметка
    batch_sz - размер одного батча
    shuffle - перемешивание элементов по батчам
    
    return:
    
    X_batches, y_batches - разделенная по батчам выборка    
    
    '''
    
    X_batches, y_batches = [], []
    
    samples_num = len(X_val)
    samples_inds = np.arange(samples_num)
    np.random.shuffle(samples_inds)
    
    cur_x, cur_y = [], []
    
    for i, cur_ind in enumerate(samples_inds):
        if i % batch_sz < batch_sz-1:
            cur_x.append(X_val[cur_ind])
            cur_y.append(y_val[cur_ind]-1)
        else:
            cur_x.append(X_val[cur_ind])
            cur_y.append(y_val[cur_ind]-1)
            X_batches.append(cur_x)
            y_batches.append(cur_y)
            cur_x, cur_y = [], []
          
    X_batches.append(cur_x)
    y_batches.append(cur_y)
    
    return np.array(X_batches), np.array(y_batches)

In [11]:
np.array(get_batch(X_test, y_test)[0][0]).shape

(16, 5, 1936)

In [0]:
from sklearn.metrics import accuracy_score, precision_score, f1_score

def get_score(preds, targets):
        
    accuracy = accuracy_score(targets, preds)
    precision_macro = precision_score(targets, preds, average='macro')
    precision_micro = precision_score(targets, preds, average='micro')
    f1_macro = f1_score(targets, preds, average='macro')
    f1_micro = f1_score(targets, preds, average='micro')
    
    return accuracy, precision_macro, precision_micro, f1_macro, f1_micro

In [0]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity as cos

In [0]:
class DSSMNet(nn.Module):
    def __init__(self, bow_size):
        super().__init__()
        self.bow_size = bow_size
        self.linear_0_1 = nn.Linear(bow_size, 300)
        self.linear_0_2 = nn.Linear(300, 300)
        self.linear_0_3 = nn.Linear(300, 128)
        
        self.linear_1_1 = nn.Linear(bow_size, 300)
        self.linear_1_2 = nn.Linear(300, 300)
        self.linear_1_3 = nn.Linear(300, 128)    
        
        self.linear_2_1 = nn.Linear(bow_size, 300)
        self.linear_2_2 = nn.Linear(300, 300)
        self.linear_2_3 = nn.Linear(300, 128)
        
        self.linear_3_1 = nn.Linear(bow_size, 300)
        self.linear_3_2 = nn.Linear(300, 300)
        self.linear_3_3 = nn.Linear(300, 128)
        
        self.linear_4_1 = nn.Linear(bow_size, 300)
        self.linear_4_2 = nn.Linear(300, 300)
        self.linear_4_3 = nn.Linear(300, 128)
        
    def forward(self, sample):
        bow_size = self.bow_size
        
        question = sample[:,0,:]
        answer1, answer2, answer3, answer4 = sample[:,1,:], sample[:,2,:], sample[:,3,:], sample[:,4,:]
        
        outputs = torch.Tensor().type(torch.cuda.FloatTensor)
        cosines = torch.Tensor().type(torch.cuda.FloatTensor)
        
        question = torch.tanh(self.linear_0_1(question))
        question = torch.tanh(self.linear_0_2(question))
        question = torch.tanh(self.linear_0_3(question))
        outputs = torch.cat((outputs, question.unsqueeze(0)), dim=0)
        
        answer1 = torch.tanh(self.linear_1_1(answer1))
        answer1 = torch.tanh(self.linear_1_2(answer1))
        answer1 = torch.tanh(self.linear_1_3(answer1))
        outputs = torch.cat((outputs, answer1.unsqueeze(0)), dim=0)
        
        answer2 = torch.tanh(self.linear_2_1(answer2))
        answer2 = torch.tanh(self.linear_2_2(answer2))
        answer2 = torch.tanh(self.linear_2_3(answer2))
        outputs = torch.cat((outputs, answer2.unsqueeze(0)), dim=0)
        
        answer3 = torch.tanh(self.linear_3_1(answer3))
        answer3 = torch.tanh(self.linear_3_2(answer3))
        answer3 = torch.tanh(self.linear_3_3(answer3))
        outputs = torch.cat((outputs, answer3.unsqueeze(0)), dim=0)
        
        answer4 = torch.tanh(self.linear_4_1(answer4))
        answer4 = torch.tanh(self.linear_4_2(answer4))
        answer4 = torch.tanh(self.linear_4_3(answer4))
        outputs = torch.cat((outputs, answer4.unsqueeze(0)), dim=0)
        
        
        for i in range(1,len(outputs)):
            cosines = torch.cat((cosines, F.cosine_similarity(outputs[0], outputs[i], dim=1).unsqueeze(0)))
            
        probs = F.softmax(cosines.view(-1,4))
        
        return probs     

In [0]:
def train(model, optimizer, X_train, y_train, batch_size):
    train_loss, all_preds, all_tests = [], [], []
    model.train()
    X_batches, y_batches = get_batch(X_train, y_train, batch_size)
    
    for batch in zip(X_batches, y_batches):
        X_batch, y_batch = batch
        X_batch, y_batch = np.array(X_batch), np.array(y_batch)
        X_batch = torch.from_numpy(X_batch).type(torch.cuda.FloatTensor)
        y_batch = torch.from_numpy(y_batch).type(torch.cuda.LongTensor)
        
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        
        train_loss.append(loss.item())
        
        max_ind = y_pred.max(1, keepdim=True)[1]
        y_pred = max_ind.view(1,-1)[0]
        all_preds.extend(y_pred.detach().cpu().numpy())
        all_tests.extend(y_batch.cpu().numpy())
        
        loss.backward()
        optimizer.step()
        
    score = get_score(np.array(all_preds), np.array(all_tests))
        
    return np.mean(train_loss), list(score)

In [0]:
def validate(model, optimizer, X_test, y_test, batch_size):
    val_loss, all_preds, all_tests = [], [], []
    model.eval()
    X_batches, y_batches = get_batch(X_test, y_test, batch_size)
    
    for batch in zip(X_batches, y_batches):
        X_batch, y_batch = batch
        X_batch, y_batch = np.array(X_batch), np.array(y_batch)
        X_batch = torch.from_numpy(X_batch).type(torch.cuda.FloatTensor)
        y_batch = torch.from_numpy(y_batch).type(torch.cuda.LongTensor)
        
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        
        val_loss.append(loss.item())
        
        max_ind = y_pred.max(1, keepdim=True)[1]
        y_pred = max_ind.view(1,-1)[0]
        all_preds.extend(y_pred.detach().cpu().numpy())
        all_tests.extend(y_batch.cpu().numpy())
        
    score = get_score(np.array(all_preds), np.array(all_tests))
        
    return np.mean(val_loss), list(score)

In [0]:
results_dict = {'Все данные': [], 'История': [], 'Медицина': [], 'Биология': []}

In [0]:
model = DSSMNet(hashes.shape[2])
model.cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
epochs = 30

In [24]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
all_train_loss, all_val_loss = [], []
all_train_acc, all_val_acc = [], []
all_scores = []
max_acc = 0

for epoch in pb(range(epochs)):
    cur_train = train(model, optimizer, X_train, y_train, 32)
    cur_val = validate(model, optimizer, X_test, y_test, 16)
    all_train_loss.append(cur_train[0])
    all_train_acc.append(cur_train[1][0])
    all_val_loss.append(cur_val[0])
    all_val_acc.append(cur_val[1][0])
    all_scores.append(cur_val[1])
    max_acc = cur_val[1][0] if cur_val[1][0] > max_acc else max_acc
    clear_output()
    print('Epoch {}:'.format(epoch+1))
    print("Train loss: {}".format(cur_train[0]))
    print("Validation loss: {}".format(cur_val[0]))
    print("Train accuracy: {}".format(cur_train[1][0]))
    print("Validation accuracy: {}".format(cur_val[1][0]))
    print("Max validation accuracy: {}".format(max_acc))

    '''f, axarr = plt.subplots(1,2, figsize=(20,10))

    axarr[0].plot(all_train_loss, label='Train loss')
    axarr[0].plot(all_val_loss, label='Val loss')
    axarr[0].grid()
    axarr[0].legend(fontsize=15)
    axarr[0].set_title('Loss after epoch #{}'.format(epoch+1), size=15)

    axarr[1].plot(all_train_acc, label='Train accuracy')
    axarr[1].plot(all_val_acc, label='Val accuracy')
    axarr[1].grid()
    axarr[1].legend(fontsize=15)
    axarr[1].set_title('Accuracy after epoch #{}'.format(epoch+1), size=15)
    plt.show()'''

results_dict['Все данные'] = np.max(np.array(all_scores), axis=0).tolist()


100% (30 of 30) |########################| Elapsed Time: 0:01:52 Time:  0:01:52


Epoch 30:
Train loss: 1.3872065738531258
Validation loss: 1.386392950462306
Train accuracy: 0.24687439892286978
Validation accuracy: 0.24026535910008653
Max validation accuracy: 0.26535910008653013


## История

In [31]:
hist_inds = []

for i in range(len(df)):
  if df.iloc[i,6] == 'История':
    hist_inds.append(i)
len(hist_inds)

2498

In [0]:
hashes_hist = hashes[hist_inds]
target = list(df[df['label'] == 'История']['5'])

X_train, X_test, y_train, y_test = TTS(hashes_hist, target, test_size=0.25, random_state=4)

In [0]:
model = DSSMNet(hashes_hist.shape[2])
model.cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
epochs = 30

In [46]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
all_train_loss, all_val_loss = [], []
all_train_acc, all_val_acc = [], []
all_scores = []
max_acc = 0

for epoch in pb(range(epochs)):
    cur_train = train(model, optimizer, X_train, y_train, 32)
    cur_val = validate(model, optimizer, X_test, y_test, 16)
    all_train_loss.append(cur_train[0])
    all_train_acc.append(cur_train[1][0])
    all_val_loss.append(cur_val[0])
    all_val_acc.append(cur_val[1][0])
    all_scores.append(cur_val[1])
    max_acc = cur_val[1][0] if cur_val[1][0] > max_acc else max_acc
    clear_output()
    print('Epoch {}:'.format(epoch+1))
    print("Train loss: {}".format(cur_train[0]))
    print("Validation loss: {}".format(cur_val[0]))
    print("Train accuracy: {}".format(cur_train[1][0]))
    print("Validation accuracy: {}".format(cur_val[1][0]))
    print("Max validation accuracy: {}".format(max_acc))

    '''f, axarr = plt.subplots(1,2, figsize=(20,10))

    axarr[0].plot(all_train_loss, label='Train loss')
    axarr[0].plot(all_val_loss, label='Val loss')
    axarr[0].grid()
    axarr[0].legend(fontsize=15)
    axarr[0].set_title('Loss after epoch #{}'.format(epoch+1), size=15)

    axarr[1].plot(all_train_acc, label='Train accuracy')
    axarr[1].plot(all_val_acc, label='Val accuracy')
    axarr[1].grid()
    axarr[1].legend(fontsize=15)
    axarr[1].set_title('Accuracy after epoch #{}'.format(epoch+1), size=15)
    plt.show()'''

results_dict['История'] = np.max(np.array(all_scores), axis=0).tolist()


100% (30 of 30) |########################| Elapsed Time: 0:00:21 Time:  0:00:21


Epoch 30:
Train loss: 1.3869654869629164
Validation loss: 1.3875593304634095
Train accuracy: 0.2402562733582488
Validation accuracy: 0.2752
Max validation accuracy: 0.2752


## Медицина

In [48]:
med_inds = []

for i in range(len(df)):
  if df.iloc[i,6] == 'Медицина':
    med_inds.append(i)
len(med_inds)

4013

In [0]:
hashes_med = hashes[med_inds]
target = list(df[df['label'] == 'Медицина']['5'])

X_train, X_test, y_train, y_test = TTS(hashes_med, target, test_size=0.25, random_state=4)

In [0]:
model = DSSMNet(hashes_med.shape[2])
model.cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
epochs = 30

In [54]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
all_train_loss, all_val_loss = [], []
all_train_acc, all_val_acc = [], []
all_scores = []
max_acc = 0

for epoch in pb(range(epochs)):
    cur_train = train(model, optimizer, X_train, y_train, 32)
    cur_val = validate(model, optimizer, X_test, y_test, 16)
    all_train_loss.append(cur_train[0])
    all_train_acc.append(cur_train[1][0])
    all_val_loss.append(cur_val[0])
    all_val_acc.append(cur_val[1][0])
    all_scores.append(cur_val[1])
    max_acc = cur_val[1][0] if cur_val[1][0] > max_acc else max_acc
    clear_output()
    print('Epoch {}:'.format(epoch+1))
    print("Train loss: {}".format(cur_train[0]))
    print("Validation loss: {}".format(cur_val[0]))
    print("Train accuracy: {}".format(cur_train[1][0]))
    print("Validation accuracy: {}".format(cur_val[1][0]))
    print("Max validation accuracy: {}".format(max_acc))

    '''f, axarr = plt.subplots(1,2, figsize=(20,10))

    axarr[0].plot(all_train_loss, label='Train loss')
    axarr[0].plot(all_val_loss, label='Val loss')
    axarr[0].grid()
    axarr[0].legend(fontsize=15)
    axarr[0].set_title('Loss after epoch #{}'.format(epoch+1), size=15)

    axarr[1].plot(all_train_acc, label='Train accuracy')
    axarr[1].plot(all_val_acc, label='Val accuracy')
    axarr[1].grid()
    axarr[1].legend(fontsize=15)
    axarr[1].set_title('Accuracy after epoch #{}'.format(epoch+1), size=15)
    plt.show()'''

results_dict['Медицина'] = np.max(np.array(all_scores), axis=0).tolist()


100% (30 of 30) |########################| Elapsed Time: 0:00:33 Time:  0:00:33


Epoch 30:
Train loss: 1.38766619029798
Validation loss: 1.3864859645328824
Train accuracy: 0.256563642406115
Validation accuracy: 0.26294820717131473
Max validation accuracy: 0.27191235059760954


## Биология

In [55]:
bio_inds = []

for i in range(len(df)):
  if df.iloc[i,6] == 'Биология':
    bio_inds.append(i)
len(bio_inds)

2184

In [0]:
hashes_bio = hashes[bio_inds]
target = list(df[df['label'] == 'Биология']['5'])

X_train, X_test, y_train, y_test = TTS(hashes_bio, target, test_size=0.25, random_state=4)

In [0]:
model = DSSMNet(hashes_bio.shape[2])
model.cuda()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()
epochs = 30

In [58]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
all_train_loss, all_val_loss = [], []
all_train_acc, all_val_acc = [], []
all_scores = []
max_acc = 0

for epoch in pb(range(epochs)):
    cur_train = train(model, optimizer, X_train, y_train, 32)
    cur_val = validate(model, optimizer, X_test, y_test, 16)
    all_train_loss.append(cur_train[0])
    all_train_acc.append(cur_train[1][0])
    all_val_loss.append(cur_val[0])
    all_val_acc.append(cur_val[1][0])
    all_scores.append(cur_val[1])
    max_acc = cur_val[1][0] if cur_val[1][0] > max_acc else max_acc
    clear_output()
    print('Epoch {}:'.format(epoch+1))
    print("Train loss: {}".format(cur_train[0]))
    print("Validation loss: {}".format(cur_val[0]))
    print("Train accuracy: {}".format(cur_train[1][0]))
    print("Validation accuracy: {}".format(cur_val[1][0]))
    print("Max validation accuracy: {}".format(max_acc))

    '''f, axarr = plt.subplots(1,2, figsize=(20,10))

    axarr[0].plot(all_train_loss, label='Train loss')
    axarr[0].plot(all_val_loss, label='Val loss')
    axarr[0].grid()
    axarr[0].legend(fontsize=15)
    axarr[0].set_title('Loss after epoch #{}'.format(epoch+1), size=15)

    axarr[1].plot(all_train_acc, label='Train accuracy')
    axarr[1].plot(all_val_acc, label='Val accuracy')
    axarr[1].grid()
    axarr[1].legend(fontsize=15)
    axarr[1].set_title('Accuracy after epoch #{}'.format(epoch+1), size=15)
    plt.show()'''

results_dict['Биология'] = np.max(np.array(all_scores), axis=0).tolist()


100% (30 of 30) |########################| Elapsed Time: 0:00:18 Time:  0:00:18


Epoch 30:
Train loss: 1.3871575754422407
Validation loss: 1.3861306462969099
Train accuracy: 0.2503052503052503
Validation accuracy: 0.27106227106227104
Max validation accuracy: 0.2967032967032967


In [59]:
results = pd.DataFrame(data=results_dict, columns=list(results_dict.keys()), \
                       index=['Accuracy', 'Precision Macro', 'Precision Micro', 'F1 Macro', 'F1 Micro'])
results

,Все данные,История,Медицина,Биология
Accuracy,0.265359,0.275200,0.271912,0.296703
Precision Macro,0.265455,0.272430,0.273681,0.296218
Precision Micro,0.265359,0.275200,0.271912,0.296703
F1 Macro,0.265134,0.272605,0.270291,0.291633
F1 Micro,0.265359,0.275200,0.271912,0.296703
